# Парсинг данных с сайта **avito.ru**

In [1]:
from bs4 import BeautifulSoup
from tqdm import tqdm
from time import sleep
import requests
import random
from urllib.parse import quote
import os
import re
import pandas as pd

## Подготовка

Для того, чтобы сайт не мог так просто заподозрить, что мы берем данные с него, возьмем заголовки с реального браузера.

In [2]:
with open('user_agents.txt', 'r') as parse_items:
    user_agents = list(map(lambda x: x.strip(), parse_items.readlines()))


Указываем ссылку, с которой будем брать данные `data_url` и ссылку-якорь, которая позволит создавать ссылки на страницы с объявлениями `base_url`.

In [3]:
# для москвы
data_url = 'https://www.avito.ru/moskva/kvartiry/prodam-ASgBAgICAUSSA8YQ'
# для всех регионов
# data_url = 'https://www.avito.ru/all/kvartiry/prodam-ASgBAgICAUSSA8YQ'

base_url = 'https://www.avito.ru'


cookie = 'личная кука'
dataset_size = 4000
links = set()


folder_to_save_csvs = 'new_csvs'
folder_to_save_htmls = 'new_html_pages'


## Приступаем к делу (парсим ссылки на объявления)

Для начала нам придется достать все ссылки с объявлениями

In [4]:
os.path.exists(folder_to_save_htmls) or os.makedirs(folder_to_save_htmls)
os.path.exists(folder_to_save_csvs) or os.makedirs(folder_to_save_csvs)
print('Созданы папки для сохранения данных')


Созданы папки для сохранения данных


In [5]:
pbar = tqdm(total=dataset_size, desc='Links obtained')

current_page = 1
while len(links) < dataset_size:

    page = requests.get(
        data_url,
        
        params={'p': current_page},
        headers={
            'User-Agent': quote(random.choice(user_agents)),

            
            'Cookie': cookie,
        }
    )

    soup = BeautifulSoup(page.text, 'html.parser')
    # print(soup.prettify())
    # break
    prev_len = len(links)

    for item in soup.find_all(name='a', attrs={'data-marker': 'item-title'}):
        if str(item.get('href')).startswith('http'):
            continue

        
        cur_link = f'{base_url}{item.get("href")}'.strip()

        
        if cur_link in links:
            continue

        links.add(cur_link)

        if len(links) >= dataset_size:
            break

    pbar.update(len(links) - prev_len)
    current_page += 1

    
    sleep(3 + abs(random.uniform(0, 1)))

del pbar

Links obtained: 100%|██████████| 4000/4000 [09:20<00:00,  7.13it/s]


In [6]:
links = list(links)


После того, как все ссылки получены, записываем на в файл, для того, чтобы нечаянно не потерять их в будущем

In [7]:
print("Примеры ссылок:")
print(*links[:5], sep='\n')
print()
print("Всего ссылок {}".format(len(links)))

with open(f'{folder_to_save_csvs}/links.csv', 'w') as links_csv:
    links_csv.write('\n'.join(links))


Примеры ссылок:
https://www.avito.ru/moskva/kvartiry/3-k._kvartira_61m_319et._3058950582
https://www.avito.ru/moskva/kvartiry/1-k._kvartira_26m_89et._3020721837
https://www.avito.ru/moskva/kvartiry/kvartira-studiya_14m_15et._2965582890
https://www.avito.ru/moskva/kvartiry/kvartira-studiya_138m_19et._2939136816
https://www.avito.ru/moskva/kvartiry/5-k._kvartira_168m_35et._2901413763

Всего ссылок 4000


## Достаем объявления

После того, как мы получили все ссылки на объявления, приступим к парсингу самих объявлений

Небольшая фукнция, которая проверяет наличие необходимых тегов на странице

In [8]:
def parse_items(x: str):
    if x is None:
        return False
    return 'params-paramsList__item' in x \
        or 'style-item-params-list-item' in x
    # or 'style-item-address__string' in x \
    # or 'style-item-map-wrapper' in x and 'style-expanded' in x


In [9]:
pbar = tqdm(total=len(links), desc='Links obtained')

for index, link in enumerate(links):
    res = requests.get(
        url=link,
        headers={
            'User-Agent': quote(random.choice(user_agents)),

            
            'Cookie': cookie,
        }
    )

    soup = BeautifulSoup(res.text)

    parsed_data = soup.find_all(
        name='li',
        class_=lambda x: parse_items(x)
    )

    if len(parsed_data) == 0:
        print(f'Что то пошло не так со ссылкой {index}\n{link}')
        with open(f'{folder_to_save_csvs}/trouble_links.txt', 'a') as trouble_links:
            trouble_links.write(link + '\n')

        continue

    filename = f"{folder_to_save_htmls}/{link.split('/')[-1]}.html"
    with open(filename, 'w') as html_file:
        html_file.write(res.text)

    with open(f'{folder_to_save_csvs}/data.csv', 'a') as output_csv:
        output_csv.write(f'{link},{filename}\n')

    pbar.update()
    sleep(3 + abs(random.uniform(0, 1)))

Links obtained:  12%|█▏        | 485/4000 [35:53<4:27:07,  4.56s/it]

Что то пошло не так со ссылкой 485
https://www.avito.ru/moskva/kvartiry/2-k._kvartira_539m_1316et._3077850052


Links obtained:  13%|█▎        | 519/4000 [38:28<4:12:02,  4.34s/it]

Что то пошло не так со ссылкой 520
https://www.avito.ru/moskva/kvartiry/2-k._apartamenty_565m_1928et._2831470833


Links obtained:  17%|█▋        | 673/4000 [50:28<4:25:00,  4.78s/it] 

...

Что то пошло не так со ссылкой 2332
https://www.avito.ru/moskva/kvartiry/2-k._kvartira_96m_1218et._3186521726


Links obtained:  61%|██████    | 2431/4000 [3:02:13<1:57:03,  4.48s/it]

Что то пошло не так со ссылкой 2443
https://www.avito.ru/moskva/kvartiry/3-k._kvartira_641m_316et._2962726010


Links obtained:  63%|██████▎   | 2534/4000 [3:09:50<1:42:00,  4.17s/it]

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

## Парсим страницы

Теперь, когда у нас есть все необходимое, можно начать работать с данными.

Напишем функцию, превращающую html страничку в словарь с данными

In [10]:

def parse_html(filename: str):

    with open(filename) as html_file:
        soup = BeautifulSoup(html_file.read())

    parsed_data = soup.find_all(
        name='li',
        class_=lambda x: parse_items(x)
    )

    if not len(parsed_data):
        return None

    parsed_data[0] = re.sub('<style.*<\/style>', '', str(parsed_data[0]))

    price = float(soup.find(name='span', attrs={
                  'itemprop': 'price'}).attrs['content'])
    currency = soup.find(name='span', attrs={
                         'itemprop': 'priceCurrency'}).attrs['content']

    location = soup.find(
        name='span', class_=lambda x: False if x is None else 'style-item-address__string-' in x)

    parsed_data = {x.contents[0].contents[0]: x.contents[1]
                   for x in parsed_data[1:]}
    parsed_data['Цена'] = price
    parsed_data['Валюта'] = currency
    if len(location.contents) < 1:
        parsed_data['Местоположение'] = None
    else:
        parsed_data['Местоположение'] = location.contents[0]

    return parsed_data


parse_html(
    f'{folder_to_save_htmls}/{links[0].split("/")[-1]}.html')

{'Общая площадь': '61\xa0м²',
 'Площадь кухни': '8\xa0м²',
 'Жилая площадь': '45\xa0м²',
 'Этаж': '3 из 19',
 'Балкон или лоджия': 'балкон',
 'Тип комнат': 'изолированные',
 'Санузел': 'раздельный',
 'Окна': 'во двор, на улицу',
 'Ремонт': 'евро',
 'Мебель': 'кухня, хранение одежды, спальные места',
 'Техника': 'кондиционер, холодильник, стиральная машина, посудомоечная машина, водонагреватель',
 'Способ продажи': 'свободная',
 'Вид сделки': 'возможна ипотека',
 'Тип дома': 'панельный',
 'Этажей в доме': '19',
 'Пассажирский лифт': '1',
 'Грузовой лифт': '1',
 'В доме': 'консьерж, мусоропровод',
 'Двор': 'закрытая территория, детская площадка, спортивная площадка',
 'Парковка': 'открытая во дворе, за шлагбаумом во дворе',
 'Цена': 11990000.0,
 'Валюта': 'RUB',
 'Местоположение': 'Москва, Юровская ул., 19'}

Получим подобные словари для всех наших данных

In [11]:
all_dicts = []
with open(f'{folder_to_save_csvs}/data.csv') as data_csv:
    for line in data_csv.readlines():
        link, filename = line.strip().split(',')

        cur_dict = parse_html(filename)
        if cur_dict is None:
            continue

        cur_dict['Ссылка'] = link

        all_dicts.append(cur_dict)

for dict_ in all_dicts[:5]:
    print(dict_)

{'Общая площадь': '61\xa0м²', 'Площадь кухни': '8\xa0м²', 'Жилая площадь': '45\xa0м²', 'Этаж': '3 из 19', 'Балкон или лоджия': 'балкон', 'Тип комнат': 'изолированные', 'Санузел': 'раздельный', 'Окна': 'во двор, на улицу', 'Ремонт': 'евро', 'Мебель': 'кухня, хранение одежды, спальные места', 'Техника': 'кондиционер, холодильник, стиральная машина, посудомоечная машина, водонагреватель', 'Способ продажи': 'свободная', 'Вид сделки': 'возможна ипотека', 'Тип дома': 'панельный', 'Этажей в доме': '19', 'Пассажирский лифт': '1', 'Грузовой лифт': '1', 'В доме': 'консьерж, мусоропровод', 'Двор': 'закрытая территория, детская площадка, спортивная площадка', 'Парковка': 'открытая во дворе, за шлагбаумом во дворе', 'Цена': 11990000.0, 'Валюта': 'RUB', 'Местоположение': 'Москва, Юровская ул., 19', 'Ссылка': 'https://www.avito.ru/moskva/kvartiry/3-k._kvartira_61m_319et._3058950582'}
{'Общая площадь': '26\xa0м²', 'Площадь кухни': '6\xa0м²', 'Этаж': '8 из 9', 'Санузел': 'совмещенный', 'Окна': 'на улиц

## Сохраняем данные

In [12]:
df = pd.DataFrame(all_dicts)
df.head()

,Общая площадь,Площадь кухни,Жилая площадь,Этаж,Балкон или лоджия,Тип комнат,Санузел,Окна,Ремонт,Мебель,...,Год постройки,Высота потолков,Тёплый пол,Запланирован снос,Отделка,Название новостройки,"Корпус, строение",Официальный застройщик,Тип участия,Срок сдачи
0,61 м²,8 м²,45 м²,3 из 19,балкон,изолированные,раздельный,"во двор, на улицу",евро,"кухня, хранение одежды, спальные места",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,26 м²,6 м²,NaN,8 из 9,NaN,NaN,совмещенный,"на улицу, на солнечную сторону",косметический,"кухня, хранение одежды, спальные места",...,1975,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,14 м²,NaN,12 м²,1 из 5,NaN,NaN,совмещенный,во двор,требует ремонта,NaN,...,1964,2.6 м,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,13.8 м²,NaN,NaN,1 из 9,NaN,NaN,совмещенный,во двор,требует ремонта,NaN,...,1971,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,168 м²,15 м²,105 м²,3 из 5,лоджия,изолированные,раздельный,во двор,дизайнерский,"кухня, хранение одежды, спальные места",...,1998,3.1 м,есть,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
df.to_csv(f'{folder_to_save_csvs}/flats.csv', index=False, sep='^')